In [20]:
# Data import and libraries

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import joblib
%matplotlib inline
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        if len(dirname) >=6 and   dirname[3:6] != 'git':
            #print(os.path.join(dirname, filename))
            pass
            

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

train=pd.read_csv("./home-data-for-ml-course/train.csv")
test=pd.read_csv("./home-data-for-ml-course/test.csv")
y_train=train['SalePrice']
X_train = train.drop('SalePrice', axis=1)

#print(test.columns)
#test already has 'SalePrice' Dropped
X_test = test

In [21]:
#Save_figs create_submission and paths
IMAGES_PATH = './output/images'
MODELS_PATH = './output/models/'
SUBMISSIONS_PATH = '.Output/Submissions/'
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

def create_submission_from_pickled(pickled, output_file='out.csv'):
    model = joblib.load(pickled)

    y_pred=model.predict(X_test)
    
    df_out = pd.DataFrame({'Id':X_test.Id, 'SalePrice': y_pred})
    out_file = os.path.join(SUBMISSIONS_PATH, output_file)
    df_out.to_csv(out_file ,index=False)

In [22]:
## Check for na categoricals
categoricals = ['MSSubClass','MSZoning', 'Street', 'Alley', 'LotShape','LandContour', 'Utilities', 'LotConfig' , 'LandSlope'
               ,'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond','RoofStyle','RoofMatl',
               'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
               'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
               'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 
               'PoolQC','Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

#Check for na categoricals
for cat in categoricals:
    nan_mean = train[train[cat].isna()]['SalePrice'].mean()
    if not pd.isna(nan_mean):
        print(f"{cat} NA:{int(nan_mean)} not NA:{int(train[train[cat].notna()]['SalePrice'].mean())}")

Alley NA:183452 not NA:142845
MasVnrType NA:236484 not NA:180615
BsmtQual NA:105652 not NA:182878
BsmtCond NA:105652 not NA:182878
BsmtExposure NA:107938 not NA:182871
BsmtFinType1 NA:105652 not NA:182878
BsmtFinType2 NA:110346 not NA:182807
Electrical NA:167500 not NA:180930
FireplaceQu NA:141331 not NA:216397
GarageType NA:103317 not NA:185479
GarageFinish NA:103317 not NA:185479
GarageQual NA:103317 not NA:185479
GarageCond NA:103317 not NA:185479
PoolQC NA:180404 not NA:288138
Fence NA:187596 not NA:152912
MiscFeature NA:182046 not NA:151623


In [23]:
#preprocessor CT - impute median and zero, OneHot

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV
from sklearn.pipeline import Pipeline


categorical = ['MSSubClass','MSZoning', 'Street', 'Alley', 'LotShape','LandContour', 'Utilities', 'LotConfig' , 'LandSlope'
               ,'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond','RoofStyle','RoofMatl',
               'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
               'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
               'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 
               'PoolQC','Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

#Use ignore to prevent failures due to rare values not occurring in cross-val sets
categorical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy="constant"))
    ,('encoder',OneHotEncoder(handle_unknown='ignore'))
])


#Split numeric fields by imputation strategy - use 0 where 0 is already a sensible value
numeric = list(x for x in X_train.columns if x not in categorical and not x == 'Id')
#print(numeric)
numeric_zero = list(_ for _ in numeric if int(X_train[_].min()) == 0)
numeric_median = list(_ for _ in numeric if int(X_train[_].min()) > 0)


numeric_zero_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="constant", fill_value=0))
    ,('scaler',StandardScaler())
])

numeric_median_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
    ,('scaler',StandardScaler())
])


preprocessor = ColumnTransformer(
    transformers=[
        ('numeric_zero', numeric_zero_transformer, numeric_zero)
        ,('numeric_median', numeric_median_transformer, numeric_median)
        ,('categorical', categorical_transformer, categorical)        
    ])

In [24]:
def fields_from_pipeline(pipeline):

    out_fields=[]

    for trans,pl,fields in pipeline.named_steps['preprocessor'].transformers:
        if trans=='categorical':
            ohe = pl.named_steps['encoder']
            imp = SimpleImputer(strategy='constant')
            dummy_data = X_train[fields]
            imp.fit(dummy_data)
            dummy_data = imp.transform(dummy_data)
            ohe.fit(dummy_data)
            #print(ohe.get_feature_names(fields))
            for _ in ohe.get_feature_names(fields):
                out_fields.append(_)
            
        else:
            for _ in fields:
                out_fields.append(_)
            
    return out_fields

In [25]:
#LR 1

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


pipeline = Pipeline(steps=[
    ('preprocessor',preprocessor)
    ,('regressor', LinearRegression())
])

ohe_fields = fields_from_pipeline(pipeline)

def fit_and_score_linear(pipeline,pickle=False,pickle_filename="out.pkl", cv_in=5):
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning) #remove spam output
    print("starting model fitting")

    lr_model = pipeline.fit(X_train, y_train)
    if pickle:
        out_file = "./Output/models/" + pickle_filename
        joblib.dump(lr_model, out_file)
        print(f"Model pickled to {out_file}" )
    #print(lr_model)

    from sklearn.metrics import mean_squared_error, make_scorer

    print("starting prediction")

    y_pred=lr_model.predict(X_train)

    print(f" prediction RMSE {mean_squared_error(y_train, y_pred) ** 0.5}")
    print(f"Coeff of determination {r2_score(y_train,y_pred)}")

    from sklearn.model_selection import cross_val_score


    print("cross-field validation")
    scores = cross_val_score(pipeline, X_train, y_train, cv=cv_in, scoring="neg_mean_squared_error")
    print(f"RMSE: {(-scores)**0.5}")
    print(f"average RMSE = {np.average((-scores)**0.5)}")
    return

fit_and_score_linear(pipeline,True,"lr1.pkl")

starting model fitting
Model pickled to ./Output/models/lr1.pkl
starting prediction
 prediction RMSE 19697.401570396356
Coeff of determination 0.9384809521716408
cross-field validation
RMSE: [29507.45887047 33570.81857236 38737.85368048 23492.00528779
 48840.51501238]
average RMSE = 34829.73028469463


In [26]:
#LR 2
from sklearn.preprocessing import PolynomialFeatures
pipeline_lr2 = Pipeline(steps=[
    ('preprocessor',preprocessor)
    ,('poly', PolynomialFeatures(degree=2))
    ,('regressor', LinearRegression())
])

fit_and_score_linear(pipeline_lr2, True, "lr2.pkl")

starting model fitting
Model pickled to ./Output/models/lr2.pkl
starting prediction
 prediction RMSE 0.31559339571007977
Coeff of determination 0.9999999999842076
cross-field validation
RMSE: [26716.20742234 48896.43053442 30780.47280572 25933.20396273
 32833.62643916]
average RMSE = 33031.988232872616


## M-encoding on Neighborhood:

In [27]:
# replacing Neighborhood with estimate of the mean
from category_encoders import MEstimateEncoder
#encodes the category with (n/n+m)*category mean + (1-n/n+m)*overall mean
#will need to remove from categorical

categorical_no_neighborhood = ['MSSubClass','MSZoning', 'Street', 'Alley', 'LotShape','LandContour', 'Utilities', 'LotConfig' , 'LandSlope'
               , 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond','RoofStyle','RoofMatl',
               'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
               'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
               'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 
               'PoolQC','Fence', 'MiscFeature', 'SaleType', 'SaleCondition']

neighborhood=['Neighborhood']

#Use ignore to prevent failures due to rare values not occurring in cross-val sets
categorical_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy="constant"))
    ,('encoder',OneHotEncoder(handle_unknown='ignore'))
])

#M=1 by default
neighborhood_transformer = Pipeline(steps=[
    ('encoder',MEstimateEncoder()) 
])

numeric = list(x for x in X_train.columns if x not in categorical and not x == 'Id' and x not in neighborhood)
numeric_zero = list(_ for _ in numeric if int(X_train[_].min()) == 0)
numeric_median = list(_ for _ in numeric if int(X_train[_].min()) > 0)


numeric_zero_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy="constant", fill_value=0))
    ,('scaler',StandardScaler())
])

numeric_median_transformer=Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
    ,('scaler',StandardScaler())
])


preprocessor_M = ColumnTransformer(
    transformers=[
        ('numeric_zero', numeric_zero_transformer, numeric_zero)
        ,('numeric_median', numeric_median_transformer, numeric_median)
        ,('neighborhood', neighborhood_transformer, neighborhood)
        ,('categorical', categorical_transformer, categorical_no_neighborhood)        
    ])

In [28]:
pipeline_M = Pipeline(steps=[
    ('preprocessor',preprocessor_M)
    ,('regressor', LinearRegression())
])

fit_and_score_linear(pipeline_M,True,'lr1_M.pkl')

starting model fitting
Model pickled to ./Output/models/lr1_M.pkl
starting prediction
 prediction RMSE 20316.577045346705
Coeff of determination 0.9345525385457886
cross-field validation
RMSE: [28812.50816355 33009.07942442 38555.75866811 23669.83697804
 47773.77736179]
average RMSE = 34364.192119183994


In [29]:
from sklearn.preprocessing import PolynomialFeatures
pipeline_lr2 = Pipeline(steps=[
    ('preprocessor',preprocessor_M)
    ,('poly', PolynomialFeatures(degree=2))
    ,('regressor', LinearRegression())
])

fit_and_score_linear(pipeline_lr2, True, "lr2.pkl")

starting model fitting
Model pickled to ./Output/models/lr2.pkl
starting prediction
 prediction RMSE 17992.94130177555
Coeff of determination 0.948667070045569
cross-field validation
RMSE: [25809.40495048 28081.04776376 46055.63014463 21609.18730181
 35854.59972249]
average RMSE = 31481.97397663445


M encoding neighborhood reduces error
* 34829 reduced to 34364 for degree 1
* 33031 reduced to 31481 for degree 2

# Grid search of multiple permutations of m encoding with ridge regularisation to reduce overfitting:


In [30]:

#210915 StandardScaler transformer added
M_enc_scale_transformer = Pipeline(steps=[
    ('encoder',MEstimateEncoder())
    ,('scaler',StandardScaler())
])

def create_categorical(to_M_encode):
    # to_M_encode: list of fields to be m_encoded
    # Returns
    out=[_ for _ in categorical if _ not in to_M_encode]
    return out

def fit_and_score_linear2(pipeline,pickle=False,pickle_filename="out.pkl", cv_in=5, X=X_train, y = y_train):
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning) #prevent warning spam
    #2: X and y as parameters
    print("starting model fitting")
    #print(pipeline)
    #return 1

    lr_model = pipeline.fit(X, y)
    if pickle:
        out_file = "./Output/models/" + pickle_filename
        joblib.dump(lr_model, out_file)
        print(f"Model pickled to {out_file}" )
    #print(lr_model)

    from sklearn.metrics import mean_squared_error, make_scorer

    print("starting prediction")

    y_pred=lr_model.predict(X)

    print(f" prediction RMSE {mean_squared_error(y, y_pred) ** 0.5}")
    print(f"Coeff of determination {r2_score(y,y_pred)}")

    from sklearn.model_selection import cross_val_score


    print("cross-field validation")
    scores = cross_val_score(pipeline, X, y, cv=cv_in, scoring="neg_mean_squared_error")
    print(f"RMSE: {(-scores)**0.5}")
    print(f"average RMSE = {np.average((-scores)**0.5)}")
    return np.average((-scores)**0.5)

In [31]:
def test_lr_M_candidate_poly_ridgeCV(degree,a,pickle_ID, pp):

    tlmnp_pipeline = Pipeline(steps=[
    ('preprocessor',pp)
    ,('poly', PolynomialFeatures(degree=degree))
    ,('regressor', RidgeCV(alphas=a, cv=5))
    ])

    pickle_name = f"lr{degree}MEnc{pickle_ID}_notm1_ridgeCV.pkl"
    return fit_and_score_linear2(tlmnp_pipeline,True,pickle_name,X = X_train,y = y_train)

In [12]:
def test_Menc(M_enc,a,a_id,results_dict):
    cat_list = create_categorical(M_enc)

    #TODO: need to consider imputation for M-encoded?
    numeric = list(x for x in X_train.columns if x not in categorical and not x == 'Id')
    #print(numeric)
    numeric_zero = list(_ for _ in numeric if int(X_train[_].min()) == 0)
    numeric_median = list(_ for _ in numeric if int(X_train[_].min()) > 0)

    pp = ColumnTransformer(
    transformers=[
        ('numeric_zero', numeric_zero_transformer, numeric_zero)
        ,('numeric_median', numeric_median_transformer, numeric_median)
        ,('neighborhood', M_enc_scale_transformer, M_enc) #was menc_list
        ,('categorical', categorical_transformer, cat_list)        
    ])

    #Identifier
    cand_id = a_id + "_"
    for feature in M_enc:
        cand_id=cand_id + feature[0] + feature[-1]
    cand_id = str(cand_id)

    #results_dict[cand_id] = test_lr_M_candidate_poly_ridge(2,469,cand_id,pp)
    results_dict[cand_id] = test_lr_M_candidate_poly_ridgeCV(2,a,cand_id,pp)

In [20]:
Menc_resultsCV={}

candidates = ['Neighborhood','OverallQual', 'OverallCond', 'GarageQual', 'MSZoning']

alp=[0,200,400,600]
alp_id="0-600by200"

for c in candidates:
    Menc_list = [c]
    test_Menc(Menc_list,alp,alp_id,Menc_resultsCV)

for c in candidates[1:]:
    Menc_list = ['Neighborhood']
    Menc_list.append(c)
    test_Menc(Menc_list,alp,alp_id,Menc_resultsCV)

for c in candidates[2:]:
    Menc_list = ['Neighborhood','OverallQual']
    Menc_list.append(c)
    test_Menc(Menc_list,alp,alp_id,Menc_resultsCV)

for c in candidates[3:]:
    Menc_list = ['Neighborhood','OverallQual', 'OverallCond']
    Menc_list.append(c)
    test_Menc(Menc_list,alp,alp_id, Menc_resultsCV)

for c in candidates[4:]:
    Menc_list = ['Neighborhood','OverallQual', 'OverallCond', 'GarageQual']
    Menc_list.append(c)
    test_Menc(Menc_list,alp,alp_id,Menc_resultsCV)

print(Menc_resultsCV)

starting model fitting
Model pickled to ./Output/models/lr2MEnc0-600by200_Nd_notm1_ridgeCV.pkl
starting prediction
 prediction RMSE 10012.116849472022
Coeff of determination 0.9841056216724395
cross-field validation
RMSE: [22920.80279586 38064.29144109 25908.82300432 20234.35850724
 28368.06466458]
average RMSE = 27099.268082618168
starting model fitting
Model pickled to ./Output/models/lr2MEnc0-600by200_Ol_notm1_ridgeCV.pkl
starting prediction
 prediction RMSE 11246.720534470163
Coeff of determination 0.9799440363239214
cross-field validation
RMSE: [23168.499972   41607.3115892  26847.9080757  20235.43848444
 29310.80435709]
average RMSE = 28233.992495687096
starting model fitting
Model pickled to ./Output/models/lr2MEnc0-600by200_Od_notm1_ridgeCV.pkl
starting prediction
 prediction RMSE 9843.263496688349
Coeff of determination 0.9846372151287404
cross-field validation
RMSE: [22664.22587833 41767.57912765 26956.01864044 19739.75891321
 31555.52648458]
average RMSE = 28536.62180884398


* minimum:
    * RMSE 0-600by200_NdOlOdMg': 26109.860572022084

Validating this in the cell below:
    


In [34]:
import joblib
best_model = joblib.load('./Output/models/lr2MEnc0-600by200_NdOlOdMg_notm1_ridgeCV.pkl')
print(f"Best alpha: {best_model.get_params()['regressor'].alpha_}")

test_resultsCV={}
test_alp=[best_model.get_params()['regressor'].alpha_]
test_alp_id=str(best_model.get_params()['regressor'].alpha_)

test_list = ['Neighborhood','OverallQual', 'OverallCond', 'MSZoning' ]
test_Menc(test_list,test_alp,test_alp_id,test_resultsCV)


Best alpha: 600
starting model fitting
Model pickled to ./Output/models/lr2MEnc600_NdOlOdMg_notm1_ridgeCV.pkl
starting prediction
 prediction RMSE 11251.294246004478
Coeff of determination 0.9799277206621059
cross-field validation
RMSE: [21232.44406153 36867.80849725 25870.94255976 19528.65433519
 27049.45340638]
average RMSE = 26109.860572022084
